In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from dataset import Dataset
from features import StrategiesSignals
import plotly.graph_objects as go

# Download data from binance

In [2]:
TICKER = "BTCUSDT"
data_binance = Dataset()
data = data_binance.get_data(days=90, ticker=TICKER, ts='5m')

In [3]:
data.tail(5)

,open,high,low,close,volume
time,,,,,
2021-08-29 23:45:00,48911.75,48974.75,48850.00,48859.43,87.47679
2021-08-29 23:50:00,48859.43,48930.00,48852.00,48918.77,55.33471
2021-08-29 23:55:00,48918.78,48918.78,48861.36,48892.27,53.86260
2021-08-30 00:00:00,48892.27,48892.28,48737.27,48767.83,133.45469
2021-08-30 00:05:00,48767.84,48861.63,48571.42,48705.27,336.71263


# Create target variables

In [4]:
size_body = 0.8
def find_candle(candle, size_body):
    rule_1 = (candle.get('close') - candle.get('low')) / (.001 + candle.get('high') - candle.get('low'))
    rule_2 = (candle.get('open') - candle.get('low')) / (.001 + candle.get('high') - candle.get('low'))
    rule_3 = candle.get('close') > candle.get('open')
    
    # Define rules
    bar = (
        ((candle.get('high') - candle.get('low')) > 3*(candle.get('open') - candle.get('close'))) &
        ((rule_1 > size_body) & (rule_1 < 1)) &
        ((rule_2 > size_body) & (rule_2 < 1)) &
        (rule_3))
    return 1 if bar else 0


In [5]:
# Look for the biggest volume for the last 1 hour (12 bars)
data['rolling_volume'] = data['volume'].rolling(5).max()
data[['volume', 'rolling_volume']].tail(10)

,volume,rolling_volume
time,,
2021-08-29 23:20:00,79.37921,102.21403
2021-08-29 23:25:00,96.82363,102.21403
2021-08-29 23:30:00,202.84696,202.84696
2021-08-29 23:35:00,169.28798,202.84696
2021-08-29 23:40:00,77.13982,202.84696
2021-08-29 23:45:00,87.47679,202.84696
2021-08-29 23:50:00,55.33471,202.84696
2021-08-29 23:55:00,53.86260,169.28798
2021-08-30 00:00:00,133.45469,133.45469


In [6]:
# Look for correction
data['rolling_price_min'] = data['close'].rolling(12).min()
data[['close', 'rolling_price_min']].tail(10)

,close,rolling_price_min
time,,
2021-08-29 23:20:00,49097.86,49025.01
2021-08-29 23:25:00,49032.67,49025.01
2021-08-29 23:30:00,48968.19,48968.19
2021-08-29 23:35:00,48896.30,48896.30
2021-08-29 23:40:00,48911.75,48896.30
2021-08-29 23:45:00,48859.43,48859.43
2021-08-29 23:50:00,48918.77,48859.43
2021-08-29 23:55:00,48892.27,48859.43
2021-08-30 00:00:00,48767.83,48767.83


In [7]:
data['signal'] = 0
for index, value in data.iterrows():
    # find pattern
    data.loc[index, 'signal'] = find_candle(value, size_body)    
    # find the biggest volume
    data.loc[index, 'signal_volume'] = 1 if value.get('volume') == value.get('rolling_volume') else 0
    # find correction
    data.loc[index, 'signal_correction'] = 1 if (value.get('close') / value.get('rolling_price_min')) -1 > -0.005 else 0

# Plot patterns

In [8]:
indexex = data[data['signal'] == 1].index
temp = data[data.index == indexex[1]]
fig = go.Figure(data=[go.Candlestick(x=temp.index,
                open=temp['open'],
                high=temp['high'],
                low=temp['low'],
                close=temp['close'])])

fig.show()

In [9]:
temp = data[
    (data['signal'] == 1) &
    (data['signal_volume'] == 1) &
    (data['signal_correction'] == 1)
]
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp.index, y=temp['close'], mode="markers", name='signals'))

fig.add_trace(go.Candlestick(x=data.index,
                             open=data['open'],
                             high=data['high'],
                             low=data['low'],
                             close=data['close'])
              )


fig.show()


In [10]:
# data.groupby('signal').count()
data.groupby(['signal', 'signal_correction', 'signal_volume']).count().T

signal              0                     1    
signal_correction 0.0        1.0        1.0    
signal_volume     0.0 1.0    0.0   1.0  0.0 1.0
open               10   1  20440  5180  189  47
high               10   1  20440  5180  189  47
low                10   1  20440  5180  189  47
close              10   1  20440  5180  189  47
volume             10   1  20440  5180  189  47
rolling_volume      6   1  20440  5180  189  47
rolling_price_min   0   0  20440  5180  189  47

In [14]:

# create lags
lags = range(1, 20)

# run loop to calculate target percent change
for i in lags:
    data[f'lag_{i}'] = data['close'].pct_change(-i)

# generate names of colimns
name_lags = [f'lag_{i}' for i in lags]
# Append our signals
name_lags.append('signal')
name_lags.append('signal_volume')
name_lags.append('signal_correction')
stat_table = data[name_lags].groupby(['signal', 'signal_correction', 'signal_volume']).agg(['median', 'count', 'min', 'max']).T #.plot()
stat_table.head(50)

signal                     0                                                1  \
signal_correction        0.0                     1.0                      1.0   
signal_volume            0.0       1.0           0.0          1.0         0.0   
lag_1  median      -0.000311  0.000999      0.000004    -0.000009    0.000144   
       count       10.000000  1.000000  20440.000000  5179.000000  189.000000   
       min         -0.003634  0.000999     -0.034128    -0.053270   -0.007414   
       max          0.002718  0.000999      0.028599     0.026319    0.007558   
lag_2  median      -0.001087  0.002814     -0.000010     0.000020    0.000106   
       count       10.000000  1.000000  20440.000000  5178.000000  189.000000   
       min         -0.006876  0.002814     -0.034422    -0.065178   -0.013956   
       max          0.002096  0.002814      0.037996     0.039474    0.010395   
lag_3  median      -0.001756  0.001635     -0.000061     0.000032   -0.000010   
       count       10.000000  1.000000  20439.000000  5178.000000  189.000000   
       min         -0.005345  0.001635     -0.036671    -0.074779   -0.019569   
       max          0.003380  0.001635      0.040517     0.036084    0.018076   
lag_4  median      -0.001554 -0.000970     -0.000077     0.000022   -0.000339   
       count       10.000000  1.000000  20438.000000  5178.000000  189.000000   
       min         -0.007636 -0.000970     -0.077835    -0.061383   -0.017454   
       max          0.002047 -0.000970      0.054954     0.045550    0.015666   
lag_5  median      -0.001652  0.000223     -0.000093     0.000039    0.000251   
       count       10.000000  1.000000  20437.000000  5178.000000  189.000000   
       min         -0.006645  0.000223     -0.078842    -0.054496   -0.019374   
       max          0.002491  0.000223      0.057516     0.047622    0.016496   
lag_6  median      -0.002060  0.001545     -0.000173     0.000023    0.000208   
       count       10.000000  1.000000  20436.000000  5178.000000  189.000000   
       min         -0.005555  0.001545     -0.077652    -0.058297   -0.022675   
       max          0.005262  0.001545      0.053029     0.062631    0.018039   
lag_7  median      -0.002621  0.002047     -0.000171    -0.000015    0.000237   
       count       10.000000  1.000000  20435.000000  5178.000000  189.000000   
       min         -0.006013  0.002047     -0.079890    -0.060723   -0.029235   
       max          0.005528  0.002047      0.051354     0.064737    0.021724   
lag_8  median      -0.002095  0.004130     -0.000205    -0.000043    0.000697   
       count       10.000000  1.000000  20435.000000  5177.000000  189.000000   
       min         -0.008598  0.004130     -0.081047    -0.055994   -0.029988   
       max          0.011749  0.004130      0.061806     0.060219    0.026721   
lag_9  median      -0.000418  0.006906     -0.000224    -0.000113    0.000503   
       count       10.000000  1.000000  20434.000000  5177.000000  189.000000   
       min         -0.007414  0.006906     -0.082462    -0.063149   -0.028052   
       max          0.010561  0.006906      0.061108     0.059576    0.026286   
lag_10 median       0.000693  0.007173     -0.000254    -0.000221    0.000528   
       count       10.000000  1.000000  20433.000000  5177.000000  189.000000   
       min         -0.006519  0.007173     -0.069176    -0.078088   -0.025130   
       max          0.012393  0.007173      0.062775     0.054511    0.030216   
lag_11 median       0.001886  0.013404     -0.000258    -0.000186    0.000451   
       count       10.000000  1.000000  20432.000000  5177.000000  189.000000   
       min         -0.005604  0.013404     -0.088874    -0.068606   -0.024715   
       max          0.009985  0.013404      0.068007     0.059516    0.024908   
lag_12 median       0.004633  0.010312     -0.000301    -0.000169    0.000298   
       count       10.000000  1.000000  20431.000000  5177.000000  189.000000   
       min         -0.004071  0

In [15]:
# Look for the best returns
stat_table = stat_table.rename_axis(["lvl0", "lvl1"])
stat_table.groupby(level=1).max()
stat_table.loc[stat_table.query("lvl1 == 'median'").idxmax(), :]

signal                    0                                       1          
signal_correction       0.0                 1.0                 1.0          
signal_volume           0.0       1.0       0.0       1.0       0.0       1.0
lvl0   lvl1                                                                  
lag_19 median      0.012034  0.024315 -0.000337 -0.000276  0.000771 -0.000272
       median      0.012034  0.024315 -0.000337 -0.000276  0.000771 -0.000272
lag_1  median     -0.000311  0.000999  0.000004 -0.000009  0.000144 -0.000113
lag_5  median     -0.001652  0.000223 -0.000093  0.000039  0.000251  0.000700
lag_18 median      0.009637  0.020634 -0.000339 -0.000237  0.001140  0.001747
       median      0.009637  0.020634 -0.000339 -0.000237  0.001140  0.001747

In [16]:
stat_table.tail(50)

signal                     0                                                1  \
signal_correction        0.0                     1.0                      1.0   
signal_volume            0.0       1.0           0.0          1.0         0.0   
lvl0   lvl1                                                                     
lag_7  min         -0.006013  0.002047     -0.079890    -0.060723   -0.029235   
       max          0.005528  0.002047      0.051354     0.064737    0.021724   
lag_8  median      -0.002095  0.004130     -0.000205    -0.000043    0.000697   
       count       10.000000  1.000000  20435.000000  5177.000000  189.000000   
       min         -0.008598  0.004130     -0.081047    -0.055994   -0.029988   
       max          0.011749  0.004130      0.061806     0.060219    0.026721   
lag_9  median      -0.000418  0.006906     -0.000224    -0.000113    0.000503   
       count       10.000000  1.000000  20434.000000  5177.000000  189.000000   
       min         -0.007414  0.006906     -0.082462    -0.063149   -0.028052   
       max          0.010561  0.006906      0.061108     0.059576    0.026286   
lag_10 median       0.000693  0.007173     -0.000254    -0.000221    0.000528   
       count       10.000000  1.000000  20433.000000  5177.000000  189.000000   
       min         -0.006519  0.007173     -0.069176    -0.078088   -0.025130   
       max          0.012393  0.007173      0.062775     0.054511    0.030216   
lag_11 median       0.001886  0.013404     -0.000258    -0.000186    0.000451   
       count       10.000000  1.000000  20432.000000  5177.000000  189.000000   
       min         -0.005604  0.013404     -0.088874    -0.068606   -0.024715   
       max          0.009985  0.013404      0.068007     0.059516    0.024908   
lag_12 median       0.004633  0.010312     -0.000301    -0.000169    0.000298   
       count       10.000000  1.000000  20431.000000  5177.000000  189.000000   
       min         -0.004071  0.010312     -0.090120    -0.066242   -0.025947   
       max          0.010023  0.010312      0.065090     0.055020    0.022428   
lag_13 median       0.004966  0.010370     -0.000332    -0.000174    0.000510   
       count       10.000000  1.000000  20430.000000  5177.000000  189.000000   
       min         -0.003519  0.010370     -0.088969    -0.065371   -0.028237   
       max          0.013533  0.010370      0.067557     0.052491    0.022736   
lag_14 median       0.008098  0.011637     -0.000317    -0.000050    0.000692   
       count       10.000000  1.000000  20429.000000  5177.000000  189.000000   
       min         -0.002243  0.011637     -0.092955    -0.065160   -0.037959   
       max          0.012342  0.011637      0.066295     0.051396    0.025426   
lag_15 median       0.007189  0.011674     -0.000306    -0.000097    0.000211   
       count       10.000000  1.000000  20428.000000  5177.000000  189.000000   
       min         -0.000169  0.011674     -0.094216    -0.063186   -0.036853   
       max          0.018967  0.011674      0.066230     0.056515    0.027249   
lag_16 median       0.007689  0.015190     -0.000337    -0.000168    0.000236   
       count       10.000000  1.000000  20427.000000  5177.000000  189.000000   
       min          0.001578  0.015190     -0.093485    -0.063842   -0.040342   
       max          0.022642  0.015190      0.072366     0.061619    0.034134   
lag_17 median       0.009338  0.013730     -0.000343    -0.000294    0.000477   
       count       10.000000  1.000000  20426.000000  5177.000000  189.000000   
       min          0.002655  0.013730     -0.081096    -0.097105   -0.042112   
       max          0.026177  0.013730      0.076111     0.061940    0.031545   
lag_18 median       0.009637  0.020634     -0.000339    -0.000237    0.001140   
       count       10.000000  1.000000  20425.000000  5177.000000  189.000000   
       min          0.003913  0.020634     -0.100473    -0.084032   -0.039730   
       max          0.024971  0